In [0]:
import zipfile, os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import keras
from os import listdir
from os.path import isfile, join
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [0]:
zip_ref = zipfile.ZipFile('Animal.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
base_model=MobileNet(weights='imagenet',include_top=False) 

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(120,activation='sigmoid')(x) 
x=Dense(120,activation='relu')(x) 
x=Dense(120,activation='relu')(x) 
x=Dense(120,activation='relu')(x) 
preds=Dense(2,activation='softmax')(x) 

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [0]:
model=Model(inputs=base_model.input,outputs=preds)

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [0]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) 
train_generator=train_datagen.flow_from_directory('Animal/',target_size=(224,224),color_mode='rgb',
                                                 batch_size=32,class_mode='categorical',shuffle=True)

In [0]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(generator=train_generator,steps_per_epoch=10,epochs=10)

In [0]:
def ToClass(image):
  if image.shape[-1] > 1:
    return image.argmax(axis=-1)
  else:
    return (image > 0.5).astype('int32')
def ResultClass(R):
  if(ToClass(R)==0):
    return 'cat'
  else:
    return 'dog'

In [0]:
mypath = "Animal/cats/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
# predicting images
dog_counter = 0 
cat_counter  = 0
for file in onlyfiles:
    img = image.load_img(mypath+file, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    classes = ToClass(classes)
    
    if classes == 0:
        print(file + ": " + 'cat')
        cat_counter += 1
    else:
        print(file + ": " + 'dog')
        dog_counter += 1
print("Total Dogs :",dog_counter)
print("Total Cats :",cat_counter)

In [0]:
mypath = "Animal/dogs/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
# predicting images
dog_counter = 0 
cat_counter  = 0
for file in onlyfiles:
    img = image.load_img(mypath+file, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    classes = ToClass(classes)
    
    if classes == 0:
        print(file + ": " + 'cat')
        cat_counter += 1
    else:
        print(file + ": " + 'dog')
        dog_counter += 1
print("Total Dogs :",dog_counter)
print("Total Cats :",cat_counter)

In [0]:
Catpath = 'Animal/cats/2Q== (2).jpg'
test_imageCat = image.load_img(Catpath, target_size=(224,224))
test_imageCat = image.img_to_array(test_imageCat)
test_imageCat = np.expand_dims(test_imageCat, axis = 0)
resultCat = model.predict(test_imageCat)
imgCat=mpimg.imread(Catpath)
imgplotCat = plt.imshow(imgCat)
plt.title(ResultClass(resultCat))
plt.show()

In [0]:
Dogpath = 'Animal/dogs/images (21).jpg'
test_imageDog = image.load_img(Dogpath, target_size=(224,224))
test_imageDog = image.img_to_array(test_imageDog)
test_imageDog = np.expand_dims(test_imageDog, axis = 0)
resultDog = model.predict(test_imageDog)
imgDog=mpimg.imread(Dogpath)
imgplotDog = plt.imshow(imgDog)
plt.title(ResultClass(resultDog))
plt.show()